In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importando bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [7]:
# Carregando os dados
# Baixe o data frame em: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
caminho_dados = r'C:\Users\Matheus Fabiao\Desktop\TCC-IA-MERCADO-FINANCEIRO\data\creditcard.csv'
base_dados = pd.read_csv(caminho_dados)

In [8]:
dados = base_dados.copy()

In [9]:
# Explorando os dados
dados.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [10]:
# Verificando se há valores nulos nos dados
dados.isnull().sum().any()

False

In [11]:
# Removendo a coluna 'Time', por não ser relevante
dados = dados.drop(['Time'], axis=1)

In [12]:
# Separando as transações em fraude e não fraude
nao_fraudes = dados[dados['Class']==0]
print(f'O número de transações válidas é de {len(nao_fraudes)}')

O número de transações válidas é de 284315


In [13]:
# Separando as transações em fraude e não fraude
fraudes = dados[dados['Class']==1]
print(f'O número de transações fraudulentas é de {len(fraudes)}')

O número de transações fraudulentas é de 492


In [14]:
# Criando um conjunto de validação com 15 transações de cada classe
validacao_nao_fraudes = nao_fraudes.sample(15)
validacao_fraudes = fraudes.sample(15)

In [15]:
# Concatenando os dois conjuntos de validação
validacao = pd.concat([validacao_nao_fraudes, validacao_fraudes], ignore_index=True)

In [16]:
# Removendo as transações de validação do DataFrame original
dados = dados.drop(validacao_nao_fraudes.index)
dados = dados.drop(validacao_fraudes.index)

In [17]:
# Criando o DataFrame com os valores reais, para comparar com as previsões
validacao_real = validacao.Class

In [18]:
# Remover a coluna alvo 'Class'
validacao = validacao.drop(['Class'], axis=1)

In [19]:
# Verificação da contagem das classes
# Desequilíbrio entre a minoritária e majoritária
dados['Class'].value_counts()

Class
0    284300
1       477
Name: count, dtype: int64

In [20]:
# Separando os dados
x = dados.drop(['Class'], axis=1)
y = dados['Class']

In [21]:
# Importação da biblioteca de oversampling
from imblearn.over_sampling import SMOTE

# Utilização do SMOTE para a sobreamostragem
smote = SMOTE(random_state=42)
X_reamostrado, y_reamostrado = smote.fit_resample(x, y)

In [22]:
# Verificação da contagem das classes
# Equilíbrio entre a minoritária e majoritária
y_reamostrado.value_counts()

Class
0    284300
1    284300
Name: count, dtype: int64

In [23]:
x_treino, x_teste, y_treino, y_teste = train_test_split(X_reamostrado, y_reamostrado, test_size=0.2, random_state=42)

In [24]:
modelo = LogisticRegression(max_iter=1000)
modelo.fit(x_treino, y_treino)

LogisticRegression(max_iter=1000)

In [25]:
previsao = modelo.predict(x_teste)

In [26]:
# Avaliando o modelo usando as principais métricas
precisao = precision_score(y_teste, previsao)
recall = recall_score(y_teste, previsao)
f1 = f1_score(y_teste, previsao)
acuracia = accuracy_score(y_teste, previsao)

In [27]:
# Exibindo as principais métricas
print(f'Precisão: {precisao:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'A acurácia do modelo foi de {acuracia:.2f}')

Precisão: 0.98
Recall: 0.94
F1-Score: 0.96
A acurácia do modelo foi de 0.96


In [28]:
# Fazendo previsões no conjunto de validação
predict = modelo.predict(validacao)

# Criando um DataFrame para comparar as classes reais com as previsões
dados = pd.DataFrame({'real':validacao_real, 'previsao':predict})

In [29]:
# Exibindo o DataFrame de comparação
dados

,real,previsao
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0
